In [1]:
# # install necessary packages to access google drive.
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse

# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# !mkdir -p drive
# !google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate

Selecting previously unselected package libfuse2:amd64.
(Reading database ... 26397 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.googl

In [2]:
# !ls ./drive/MSCBDT5002FINAL/

Data_Q1  Data_Q2  Data_Q4  Data_Q6  Data_Q8


In [3]:
import numpy as np
import pandas as pd
from keras.preprocessing import image

from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input as mobilenet_input_pp

import os
from sklearn.utils.random import sample_without_replacement
from matplotlib import pyplot as plt


Using TensorFlow backend.


In [0]:
data_root = './drive/MSCBDT5002FINAL/Data_Q1'
inlier_folder = 'inlier_train'
outlier_folder = 'outlier_train'
test_folder = 'test'

In [0]:
inlier_paths = []
outlier_paths = []
test_paths = []

up_folder = os.path.join(data_root, inlier_folder)
for root, folders, files in os.walk(up_folder):
    for file in files:
        inlier_paths.append(os.path.join(up_folder, file))

up_folder = os.path.join(data_root, outlier_folder)
for root, folders, files in os.walk(up_folder):
    for file in files:
        outlier_paths.append(os.path.join(up_folder, file))

up_folder = os.path.join(data_root, test_folder)
for root, folders, files in os.walk(up_folder):
    for file in files:
        test_paths.append(os.path.join(up_folder, file))

In [0]:

def gen_img_features(fe_model, fe_input_prossesor, img_path_list, resample=False, sample_size=0.3):
    
    paths = []
    if resample:
        population_size = len(img_path_list)
        sample_size = int(population_size*sample_size)
        sample_index = sample_without_replacement(n_samples=sample_size, n_population=population_size, method='auto')
        
        for i in sample_index:
            paths.append(img_path_list[i])
    else:
        paths = img_path_list
        
    img_features = []
    count = 0
    for img_path in paths:
        count += 1
        print(f'{count}th image starts.')
        
        img_data = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img_data)
        img_data = fe_input_prossesor(np.expand_dims(img_data, axis=0))

        features = fe_model.predict(img_data)
        features = np.array(features).flatten() # collapse feature into one-dim
    
        img_features.append(features)
        

    img_vgg_features = np.array(img_features)
    df_features = pd.DataFrame(img_features, columns=list(range(img_vgg_features.shape[1]))).set_index([paths])
    
    return df_features

In [7]:
print(len(test_paths)) 
print(len(inlier_paths))
print(len(outlier_paths))


1300
4801
413


In [8]:
%%time
# generate dataset firstly

fe_model = MobileNet(weights='imagenet', include_top=False, pooling='avg')
fe_model.summary()

# resample inliers set in gen_img_features() function
inlier_df = gen_img_features(fe_model, mobilenet_input_pp, inlier_paths, resample=True, sample_size=0.25)
print('inlier feature generated')
outlier_df = gen_img_features(fe_model, mobilenet_input_pp, outlier_paths, resample=False)
print('outlier feature generated')


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


17227776/17225924 [==============================] - 1s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_b

In [9]:
test_df = gen_img_features(fe_model, mobilenet_input_pp, test_paths, resample=False)

1th image starts.
2th image starts.
3th image starts.
4th image starts.
5th image starts.
6th image starts.
7th image starts.
8th image starts.
9th image starts.
10th image starts.
11th image starts.
12th image starts.
13th image starts.
14th image starts.
15th image starts.
16th image starts.
17th image starts.
18th image starts.
19th image starts.
20th image starts.
21th image starts.
22th image starts.
23th image starts.
24th image starts.
25th image starts.
26th image starts.
27th image starts.
28th image starts.
29th image starts.
30th image starts.
31th image starts.
32th image starts.
33th image starts.
34th image starts.
35th image starts.
36th image starts.
37th image starts.
38th image starts.
39th image starts.
40th image starts.
41th image starts.
42th image starts.
43th image starts.
44th image starts.
45th image starts.
46th image starts.
47th image starts.
48th image starts.
49th image starts.
50th image starts.
51th image starts.
52th image starts.
53th image starts.
54

In [0]:
# from sklearn.decomposition import PCA
# the number of feature is not too big, no need to reduce dimension

In [0]:
inlier_df['label'] = np.repeat(0, len(inlier_df))
outlier_df['label'] = np.repeat(1, len(outlier_df))

In [0]:
inlier_df.to_csv(os.path.join(data_root, 'inlier_features.csv'))
outlier_df.to_csv(os.path.join(data_root, 'outlier_features.csv'))

In [0]:
train_df = pd.DataFrame()
train_df = train_df.append(inlier_df)
train_df = train_df.append(outlier_df)

In [13]:
len(train_df)

1613

### Modeling

In [0]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


#### Validation to proper XGBoost 

In [0]:
xgb1  = XGBClassifier(booster='gbtree', n_estimators=600, learning_rate=0.1, max_depth=3)
# xgb2  = XGBClassifier(booster='gbtree', n_estimators=1000, learning_rate=0.1, max_depth=2)
# xgb3  = XGBClassifier(booster='gbtree', n_estimators=2000, learning_rate=0.1, max_depth=1)

In [0]:
train_y = train_df[['label']]
train_x = train_df.drop('label', axis=1)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2)

In [22]:
xgb1.fit(x_train, y_train)
pred_y = xgb1.predict(x_test)
print(accuracy_score(y_test, pred_y))

# xgb2.fit(x_train, y_train)
# pred_y = xgb2.predict(x_test)
# print(accuracy_score(y_test, pred_y))

# xgb3.fit(x_train, y_train)
# pred_y = xgb3.predict(x_test)
# print(accuracy_score(y_test, pred_y))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9411764705882353


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### Generate output file

In [25]:
model  = XGBClassifier(booster='gbtree', n_estimators=600, learning_rate=0.1, max_depth=3)
model.fit(train_x, train_y)
pred_y = model.predict(test_df)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [0]:
ids = [i.split('/')[-1].split('.')[0] for i in test_paths]
test_df['ID'] = np.array(ids)
test_df['Result'] = pred_y

In [0]:
test_df[['ID', 'Result']].to_csv('./drive/5002_Q1_output_V2.csv', index=False)

In [28]:
test_df[['ID', 'Result']]

,ID,Result
./drive/MSCBDT5002FINAL/Data_Q1/test/img_eccfb81367b04177a3665520ceaac422.jpg,img_eccfb81367b04177a3665520ceaac422,1
./drive/MSCBDT5002FINAL/Data_Q1/test/img_edf7354754f8408cae5111522dd969fc.jpg,img_edf7354754f8408cae5111522dd969fc,1
./drive/MSCBDT5002FINAL/Data_Q1/test/img_ec2da75bbdfe47d8b20cd2952a3e94ac.jpg,img_ec2da75bbdfe47d8b20cd2952a3e94ac,1
./drive/MSCBDT5002FINAL/Data_Q1/test/img_ef157f2662da46b1aee9b5c45e5cdc14.jpg,img_ef157f2662da46b1aee9b5c45e5cdc14,1
./drive/MSCBDT5002FINAL/Data_Q1/test/img_efc2c3c3ae4e4d688db827335f0489a8.jpg,img_efc2c3c3ae4e4d688db827335f0489a8,1
./drive/MSCBDT5002FINAL/Data_Q1/test/img_ef5a755cf8db40db8803b24d0a01856c.jpg,img_ef5a755cf8db40db8803b24d0a01856c,1
./drive/MSCBDT5002FINAL/Data_Q1/test/img_f0239b5638f04418bd2b8445a051bd3e.jpg,img_f0239b5638f04418bd2b8445a051bd3e,1
./drive/MSCBDT5002FINAL/Data_Q1/test/img_f0418ff0a9ba47ceb87516f9db65f0db.jpg,img_f0418ff0a9ba47ceb87516f9db65f0db,1
./drive/MSCBDT5002FINAL/Data_Q1/test/img_ef6b8ca9f26745b188bd32c78c5ee3e2.jpg,img_ef6b8ca9f26745b188bd32c78c5ee3e2,1
./drive/MSCBDT5002FINAL/Data_Q1/test/img_ef6765f13dc8476087dbc313cd6fe098.jpg,img_ef6765f13dc8476087dbc313cd6fe098,1
